Data Analysis on Lip Reading Dataset

In [15]:
import os
import glob
import pandas as pd
import seaborn as sns
import subprocess as sb
from collections import Counter
import json
import ast
#Phonetics Libraries:
import epitran
import gruut_ipa

In [3]:
#Dataset contains 1100 videos per word. 1000 in trai folder, 50 in test folder, and 50 in Val folder


totalDir = 0
totalFiles = 0
wordslist= []
trainset = []
testset = []
valset = []

dataset_folder = './lipread_mp4'

for base,dirs,files in os.walk(dataset_folder):
    for directories in dirs:
        totalDir += 1
        if directories == 'train':
            trainset.append(directories)
        elif directories == 'test':
            testset.append(directories)
        elif directories == 'val':
            valset.append(directories)
        else:
            wordslist.append(directories)
    for Files in files:
        totalFiles += 1

print('Total # of files :', totalFiles)
print('Toatal # of directories :', totalDir)
print('Number of words :', len(wordslist))
print('Wordlist :', wordslist)

Total # of files : 1077532
Toatal # of directories : 2000
Number of words : 500
Wordlist : ['ABOUT', 'ABSOLUTELY', 'ABUSE', 'ACCESS', 'ACCORDING', 'ACCUSED', 'ACROSS', 'ACTION', 'ACTUALLY', 'AFFAIRS', 'AFFECTED', 'AFRICA', 'AFTER', 'AFTERNOON', 'AGAIN', 'AGAINST', 'AGREE', 'AGREEMENT', 'AHEAD', 'ALLEGATIONS', 'ALLOWED', 'ALMOST', 'ALREADY', 'ALWAYS', 'AMERICA', 'AMERICAN', 'AMONG', 'AMOUNT', 'ANNOUNCED', 'ANOTHER', 'ANSWER', 'ANYTHING', 'AREAS', 'AROUND', 'ARRESTED', 'ASKED', 'ASKING', 'ATTACK', 'ATTACKS', 'AUTHORITIES', 'BECAUSE', 'BECOME', 'BEFORE', 'BEHIND', 'BEING', 'BELIEVE', 'BENEFIT', 'BENEFITS', 'BETTER', 'BETWEEN', 'BIGGEST', 'BILLION', 'BLACK', 'BORDER', 'BRING', 'BRITAIN', 'BRITISH', 'BROUGHT', 'BUDGET', 'BUILD', 'BUSINESS', 'BUSINESSES', 'CALLED', 'CAMERON', 'CAMPAIGN', 'CANCER', 'CANNOT', 'CAPITAL', 'CASES', 'CENTRAL', 'CERTAINLY', 'CHALLENGE', 'CHANCE', 'CHANGE', 'CHANGES', 'CHARGE', 'CHARGES', 'CHIEF', 'CHILD', 'CHILDREN', 'CLAIMS', 'CLEAR', 'CLOSE', 'CLOUD', 'COMES', 'C

In [60]:


#convert words to lowercase
lowercase_words = []
for single in wordslist:
    lowercase = single.lower()
    lowercase_words.append(lowercase)

allphonemes = []

epi = epitran.Epitran('eng-Latn', cedict_file='cedict_1_0_ts_utf-8_mdbg.txt')

#create dict of lists to add values as descriptions of phonemes are created
consonants = {"text": [], 'letters': [], "place": [], "voiced": []}
vowels = {'text': [], 'letters': [], 'height': [], 'placement': [], 'rounded': []}

def wordstophones(wordinsert):

    phonword = epi.transliterate(wordinsert)
    return phonword
    
def phondescribe(fullphone):

    spacedphones = sb.run(['python3 -m gruut_ipa phones ' + fullphone], shell=True, capture_output=True, text=True)

    splitphone = spacedphones.stdout.split()

    for phone in splitphone:

        allphonemes.append(phone)

        description = sb.run(['python3 -m gruut_ipa describe ' + phone], shell=True, capture_output=True, text=True)

        description = description.stdout.replace('true','True').replace('false', 'False').replace('[]', 'None').replace('""', 'None')
        
        desdict = ast.literal_eval(description)

        if desdict["type"] == "Consonant":
            for k,v in desdict.items():
                if k in consonants.keys():
                    consonants[k].append(v)
                else:
                    continue
        elif desdict["type"] == "Vowel":
            for k,v in desdict.items():
                if k in vowels.keys():
                    vowels[k].append(v)
                else:
                    continue
        else:
            print('Error: Letter type not Consonant or Vowel')
    






In [24]:
'''
TEST CELL, PLEASE IGNORE


description = sb.run(['python3 -m gruut_ipa describe ' + 'e'], shell=True, capture_output=True, text=True)

description = description.stdout.replace('true','True').replace('false', 'False').replace('[]', 'None').replace('""', 'None')



print(type(description))

print(description)

desdict = ast.literal_eval(description)
print(type(desdict))
print(desdict)
'''

<class 'str'>
{"text": "e", "letters": "e", "tone": None, "tones": None, "accents": None, "stress": None, "height": "close-mid", "placement": "front", "rounded": False, "type": "Vowel", "nasalated": None, "raised": None, "elongated": False}

<class 'dict'>
{'text': 'e', 'letters': 'e', 'tone': None, 'tones': None, 'accents': None, 'stress': None, 'height': 'close-mid', 'placement': 'front', 'rounded': False, 'type': 'Vowel', 'nasalated': None, 'raised': None, 'elongated': False}


In [61]:

for each_word in lowercase_words:
    #progress-check
    print('Current word:' + each_word)

    pronounciation = wordstophones(each_word)
    phondescribe(pronounciation)

    
    

Current word:about
Current word:absolutely
Current word:abuse
Current word:access
Current word:according
Current word:accused
Current word:across
Current word:action
Current word:actually
Current word:affairs
Current word:affected
Current word:africa
Current word:after
Current word:afternoon
Current word:again
Current word:against
Current word:agree
Current word:agreement
Current word:ahead
Current word:allegations
Current word:allowed
Current word:almost
Current word:already
Current word:always
Current word:america
Current word:american
Current word:among
Current word:amount
Current word:announced
Current word:another
Current word:answer
Current word:anything
Current word:areas
Current word:around
Current word:arrested
Current word:asked
Current word:asking
Current word:attack
Current word:attacks
Current word:authorities
Current word:because
Current word:become
Current word:before
Current word:behind
Current word:being
Current word:believe
Current word:benefit
Current word:benefits
C

In [62]:
#Construct Consonant and Vowel DataFrames
cons_dataframe = pd.DataFrame(consonants)
vow_dataframe = pd.DataFrame(vowels)

print(cons_dataframe.head())
print(vow_dataframe.head())

  text letters     place  voiced
0    b       b  bilabial    True
1    w       w  bilabial    True
2    t       t  alveolar   False
3    b       b  bilabial    True
4    s       s  alveolar   False
  text letters     height placement  rounded
0    ə       ə        mid   central    False
1    a       a       open     front    False
2    æ       æ  near-open     front    False
3    ə       ə        mid   central    False
4    u       u      close      back     True


0